In [12]:
import gurobipy as gp
from gurobipy import *
import numpy as np
import pandas as pd

In [13]:
# Dictionaries for factories (supply in tons), depots(throughput in tons), customers(demand in tons)
f = dict({
    'Liverpool': 150000,
    'Brighton': 200000
})

d = ({
    'Newcastle': 70000,
    'Birmingham': 50000,
    'London': 100000,
    'Exeter': 40000
})

c = ({
    'C1': 50000,
    'C2': 10000,
    'C3': 40000,
    'C4': 35000,
    'C5': 60000,
    'C6': 20000
})

In [14]:
#non-prefered arcs
non_prefer=[('Liverpool', 'C6'),('Brighton', 'C1'),('Newcastle', 'C5'),('Newcastle', 'C6'),('Birmingham', 'C1'),('Birmingham', 'C2'),
           ('London', 'C2'),('London', 'C5'),('Exeter', 'C5')]

In [15]:
weights=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
dict_weighted={}
df_weighted= pd.DataFrame(columns=['weights','cost function','non-prefered quantities', 'non-prefered supliers'])
for w in weights:
    # Dictionary for distribution costs per factory (in €/ton) for the sum of 2 objective functions 
    arcs, distr_cost = gp.multidict({
    ('Liverpool', 'Newcastle'): (1-w)*0.5,
    ('Liverpool', 'Birmingham'): (1-w)*0.5,
    ('Liverpool', 'London'): (1-w)*1.0,
    ('Liverpool', 'Exeter'): (1-w)*0.2,
    ('Liverpool', 'C1'): (1-w)*1.0,
    ('Liverpool', 'C3'): (1-w)*1.5,
    ('Liverpool', 'C4'): (1-w)*2.0,
    ('Liverpool', 'C6'): (1-w)*1.0+w,
    ('Brighton', 'Birmingham'): (1-w)*0.3,
    ('Brighton', 'London'): (1-w)*0.5,
    ('Brighton', 'Exeter'): (1-w)*0.2,
    ('Brighton', 'C1'): (1-w)*2.0+w,
    ('Newcastle', 'C2'): (1-w)*1.5,
    ('Newcastle', 'C3'): (1-w)*0.5,
    ('Newcastle', 'C4'): (1-w)*1.5+w,
    ('Newcastle', 'C6'): (1-w)*1.0+w,
    ('Birmingham', 'C1'): (1-w)*1.0+w,
    ('Birmingham', 'C2'): (1-w)*0.5+w,
    ('Birmingham', 'C3'): (1-w)*0.5,
    ('Birmingham', 'C4'): (1-w)*1.0,
    ('Birmingham', 'C5'): (1-w)*0.5,
    ('London', 'C2'): (1-w)*1.5+w,
    ('London', 'C3'): (1-w)*2.0,
    ('London', 'C5'): (1-w)*0.5+w,
    ('London', 'C6'): (1-w)*1.5,
    ('Exeter', 'C3'): (1-w)*0.2,
    ('Exeter', 'C4'): (1-w)*1.5,
    ('Exeter', 'C5'): (1-w)*0.5+w,
    ('Exeter', 'C6'): (1-w)*1.5    
    })
    # Define model
    model = Model('Transshipment_Assignment')

    # Create decision variables
    plan = model.addVars(arcs, name="plan")

    # set objective function
    model.setObjective(plan.prod(distr_cost), GRB.MINIMIZE)
    
    # Define constraints
    # maximal factory supply
    factories = f.keys()
    factory_con = model.addConstrs((gp.quicksum(plan.select(i, '*')) <= f[i] for i in factories), name="factory_constraint")

    # maximal depot throughput
    depots = d.keys()
    depot_through_con = model.addConstrs((gp.quicksum(plan.select(j, '*')) <= d[j] for j in depots), 
                                     name="depot_through_constraint")
    # depot output == depot input
    depot_out_con = model.addConstrs((gp.quicksum(plan.select('*', j)) == gp.quicksum(plan.select(j, '*')) for j in depots),
                                                                                 name="depot_out_constraint")
    # customer requirement == customer input
    customers = c.keys()
    customer_con = model.addConstrs((gp.quicksum(plan.select('*', k)) == c[k] for k in customers), name="customer_constraint")
    
    model.optimize()
    # Optimal value of the objective function which is the weighted sum of two objective functions (in €)
    obj = model.getObjective()
    obj_val = obj.getValue()
    # Optimal transportation plan per month (quantities in tons)
    trans_plan={}
    for a in arcs:
        if plan[a].x > 1e-6:
            trans_plan.update({(a[0], a[1]): plan[a].x})
    #calculate objective function 2 and 1, save the dict of non-prefered arcs vs quantities of goods transshipped 
    obj_2=0
    obj_1=0
    non_pref_dict={}
    for a in non_prefer:
        if a in trans_plan.keys():
            non_pref_dict.update({a:trans_plan[a]})
            obj_2 += trans_plan[a]
    obj_1=(obj_val-w*obj_2)/(1-w)
    #update result
    dict_weighted.update({w:[obj_1,obj_2,non_pref_dict]})
    df_weighted=df_weighted.append({ 'weights':w, 'cost function':obj_1, 'non-prefered quantities': obj_2, 'non-prefered supliers':non_pref_dict},ignore_index=True)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 29 columns and 75 nonzeros
Model fingerprint: 0x824994fc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+04, 2e+05]
Presolve removed 1 rows and 1 columns
Presolve time: 0.04s
Presolved: 15 rows, 28 columns, 73 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.3620000e+05   2.624375e+04   0.000000e+00      0s
       5    1.8715000e+05   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.08 seconds
Optimal objective  1.871500000e+05
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 16 rows, 29 columns and 75 nonzeros
Model fingerprint: 0x7898b240
Coefficient statistics:
  Matr

In [16]:
dict_weighted

{0.1: [198500.0,
  85000.0,
  {('Liverpool', 'C6'): 20000.0,
   ('Birmingham', 'C2'): 10000.0,
   ('London', 'C5'): 55000.0}],
 0.2: [198500.0,
  85000.0,
  {('Liverpool', 'C6'): 20000.0,
   ('Birmingham', 'C2'): 10000.0,
   ('London', 'C5'): 55000.0}],
 0.3: [198500.0,
  85000.0,
  {('Liverpool', 'C6'): 20000.0,
   ('Birmingham', 'C2'): 10000.0,
   ('London', 'C5'): 55000.0}],
 0.4: [226000.0,
  30000.0,
  {('Liverpool', 'C6'): 20000.0, ('London', 'C5'): 10000.0}],
 0.5: [231000.0,
  25000.0,
  {('Liverpool', 'C6'): 15000.0, ('London', 'C5'): 10000.0}],
 0.6: [246000.0, 10000.0, {('London', 'C5'): 10000.0}],
 0.7: [245999.99999999997, 10000.0, {('London', 'C5'): 10000.0}],
 0.8: [246000.00000000003, 10000.0, {('London', 'C5'): 10000.0}],
 0.9: [245999.99999999997, 10000.0, {('London', 'C5'): 10000.0}]}

In [17]:
df_weighted

,weights,cost function,non-prefered quantities,non-prefered supliers
0,0.1,198500.0,85000.0,"{('Liverpool', 'C6'): 20000.0, ('Birmingham', ..."
1,0.2,198500.0,85000.0,"{('Liverpool', 'C6'): 20000.0, ('Birmingham', ..."
2,0.3,198500.0,85000.0,"{('Liverpool', 'C6'): 20000.0, ('Birmingham', ..."
3,0.4,226000.0,30000.0,"{('Liverpool', 'C6'): 20000.0, ('London', 'C5'..."
4,0.5,231000.0,25000.0,"{('Liverpool', 'C6'): 15000.0, ('London', 'C5'..."
5,0.6,246000.0,10000.0,"{('London', 'C5'): 10000.0}"
6,0.7,246000.0,10000.0,"{('London', 'C5'): 10000.0}"
7,0.8,246000.0,10000.0,"{('London', 'C5'): 10000.0}"
8,0.9,246000.0,10000.0,"{('London', 'C5'): 10000.0}"


In [ ]:
# The Pareto set (non-dominated solution set) has 4 solutions: df_weighted[0], df_weighted[3], df_weighted[4],df_weighted[5].
